In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import time

In [2]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 15000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# 

In [3]:
torch.manual_seed(1337)

In [4]:
# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(f'Some characters: {chars[10:20]}')
print(f'vocab_size: {vocab_size}')

Some characters: [':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G']
vocab_size: 65


In [6]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
print(data)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

tensor([18, 47, 56,  ..., 52, 45,  8])


In [7]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [8]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        perplexities = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
            perplexities[k] = torch.exp(loss).item()
        # out[split] = losses.mean()
        out[split + '_loss'] = losses.mean()
        out[split + '_perplexity'] = perplexities.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [9]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

0.209729 M parameters


In [10]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

start_time = time.time()
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train_loss']:.4f}, val loss {losses['val_loss']:.4f}, train perplexity {losses['train_perplexity']:.4f}, val perplexity {losses['val_perplexity']:.4f}")


    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

end_time = time.time()

step 0: train loss 4.4112, val loss 4.4015, train perplexity 82.3993, val perplexity 81.5992
step 100: train loss 2.6576, val loss 2.6632, train perplexity 14.2867, val perplexity 14.3680
step 200: train loss 2.5119, val loss 2.5023, train perplexity 12.3440, val perplexity 12.2302
step 300: train loss 2.4156, val loss 2.4310, train perplexity 11.2130, val perplexity 11.3902
step 400: train loss 2.3516, val loss 2.3672, train perplexity 10.5188, val perplexity 10.6889
step 500: train loss 2.3013, val loss 2.3223, train perplexity 10.0018, val perplexity 10.2168
step 600: train loss 2.2554, val loss 2.2618, train perplexity 9.5545, val perplexity 9.6190
step 700: train loss 2.2136, val loss 2.2233, train perplexity 9.1674, val perplexity 9.2590
step 800: train loss 2.1601, val loss 2.1915, train perplexity 8.6861, val perplexity 8.9669
step 900: train loss 2.1417, val loss 2.1512, train perplexity 8.5309, val perplexity 8.6102
step 1000: train loss 2.1025, val loss 2.1319, train perplex

In [11]:
training_time = end_time - start_time

In [12]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))



YORK:
I beince.

STANLO:
God mad!
What, you eyearged.

MENENIUS:
I am but foul his vety cedilaius.

First Serven'd at my know
Yoursel'd upon your mile, will,
Whilsters, I in latest in overtue,
Which now thee abousel lind tear.

KING RICHARD II:
I am spelw you love.
I cannot to the hope.
I the goldesson,--
A what erichle, and doubt, Then, in Francous
And strume king thrust for the
grean upon thy flexol, for dear?

KING HENRY VI:
Hast you say advantagened for his business, tey, rept
force thee wilt me: anks mary thou conspant?
 so a cemnst.--'nough the miner and should consul,
And then, the much friend stoutchs lord
Is as we lastings.

LUCIO:
About oven: it pats
Lest's counsely some so upon thou fest, as not come,
Against is my side he need our tords,
Hath with were but with soul. Dove long me too welcome.

QUEEN Fivence:
The demiture powern'd.

GREMIO:
With the offinuserter now the subme will;
Good what thou to cousless Bluelike
I carry O, thy body twalk, time wout
Shall use on her ou

In [27]:
def generate_sample(model, seed_text, max_new_tokens=40):
    context = torch.tensor([encode(seed_text)], dtype=torch.long, device=device)
    return decode(model.generate(context, max_new_tokens=max_new_tokens)[0].tolist())

sample_text = generate_sample(model, "What dismal visage dost thou see")
print(sample_text)

What dismal visage dost thou sees;
And for you pasty me with ingernects!


In [28]:

print(f"Training Time: {training_time}")


Training Time: 461.4930558204651


In [29]:

print(torch.__version__)


2.3.1+cu121
